BERT EXTRCATOR CODE HAS BEEN ADAPTED FROM - https://github.com/Ighina/MultiModalSA/blob/master/utils/BERT_embeddings_extractor.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3n3bxmp7
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-3n3bxmp7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 26.6 MB/s 
     |████████████████████████████████| 895 kB 48.0 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4004577 sha256=2fcfc3d5f6031df1fb3d93cdfcd7334acd8333077b4f1acb948d4609d6590eff
  Stored in directory: /tmp/pip-ephem-wheel-cache-cthe0xsd/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uni

In [ ]:
!pip install transformers

In [ ]:
import transformers
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

import copy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(transformers.__version__)

4.19.0.dev0


call cuda

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentence-Train-Final.csv")
display(train_df.head())

,Sentence,Sentence Length
0,independence we are only bound by human rights...,19
1,sp sp sp sp we scrutinize what governments wha...,20
2,sp sp sp sp sp sp sp sp sp sp sp sp sp sp inde...,20
3,sp sp sp sp work we help the states to impleme...,20
4,us to do that effectively we need to be indepe...,20


In [ ]:
emo_df_train = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/csv_files/eTrain.csv")
display(emo_df_train.head()) 

,Anger,Disgust,Fear,Happy,Sad,Surprise
0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_df_final = pd.concat([train_df, emo_df_train], axis = 1)
display(train_df_final.head())

,Sentence,Sentence Length,Anger,Disgust,Fear,Happy,Sad,Surprise
0,independence we are only bound by human rights...,19,0.0,1.0,0.0,0.0,0.0,0.0
1,sp sp sp sp we scrutinize what governments wha...,20,1.0,0.0,0.0,0.0,0.0,0.0
2,sp sp sp sp sp sp sp sp sp sp sp sp sp sp inde...,20,1.0,0.0,0.0,0.0,0.0,0.0
3,sp sp sp sp work we help the states to impleme...,20,1.0,0.0,0.0,0.0,0.0,0.0
4,us to do that effectively we need to be indepe...,20,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_df_final.drop("Sentence Length", axis=1, inplace=True)
display(train_df_final.head())

,Sentence,Anger,Disgust,Fear,Happy,Sad,Surprise
0,independence we are only bound by human rights...,0.0,1.0,0.0,0.0,0.0,0.0
1,sp sp sp sp we scrutinize what governments wha...,1.0,0.0,0.0,0.0,0.0,0.0
2,sp sp sp sp sp sp sp sp sp sp sp sp sp sp inde...,1.0,0.0,0.0,0.0,0.0,0.0
3,sp sp sp sp work we help the states to impleme...,1.0,0.0,0.0,0.0,0.0,0.0
4,us to do that effectively we need to be indepe...,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(train_df_final.shape)

(15290, 7)


In [ ]:
valid_df = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentence-Valid-Final.csv")
display(valid_df.head())

,Sentence,Sentence Length
0,sp sp sp sp sp albums now you might wonder why...,20
1,sp sp sp sp sp sp sp sp sp sp this is another ...,20
2,albums like this that i always misplace or tha...,20
3,sp sp sp sp sp sp sp sp sp sp sp sp sp sp how ...,20
4,sp sp sp out there that will give you an oppor...,20


In [ ]:
emo_df_valid = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/csv_files/eValid.csv")
display(emo_df_valid.head()) 

,Anger,Disgust,Fear,Happy,Sad,Surprise
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
valid_df_final = pd.concat([valid_df, emo_df_valid], axis = 1)
display(valid_df_final.head())

,Sentence,Sentence Length,Anger,Disgust,Fear,Happy,Sad,Surprise
0,sp sp sp sp sp albums now you might wonder why...,20,0.0,0.0,0.0,1.0,0.0,0.0
1,sp sp sp sp sp sp sp sp sp sp this is another ...,20,0.0,0.0,0.0,0.0,1.0,0.0
2,albums like this that i always misplace or tha...,20,0.0,0.0,0.0,1.0,0.0,0.0
3,sp sp sp sp sp sp sp sp sp sp sp sp sp sp how ...,20,0.0,0.0,0.0,1.0,0.0,0.0
4,sp sp sp out there that will give you an oppor...,20,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
valid_df_final.drop("Sentence Length", axis=1, inplace=True)
display(valid_df_final.head())

,Sentence,Anger,Disgust,Fear,Happy,Sad,Surprise
0,sp sp sp sp sp albums now you might wonder why...,0.0,0.0,0.0,1.0,0.0,0.0
1,sp sp sp sp sp sp sp sp sp sp this is another ...,0.0,0.0,0.0,0.0,1.0,0.0
2,albums like this that i always misplace or tha...,0.0,0.0,0.0,1.0,0.0,0.0
3,sp sp sp sp sp sp sp sp sp sp sp sp sp sp how ...,0.0,0.0,0.0,1.0,0.0,0.0
4,sp sp sp out there that will give you an oppor...,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
print(valid_df_final.shape)

(2291, 7)


In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/data/rawSentence-Test-Final.csv")
display(test_df.head())

,Sentence,Sentence Length
0,what doing sp not talking about just going int...,19
1,super connectors that help businesses via thei...,20
2,sp sp sp sp now you see this online with big b...,20
3,sp sp sp sp sp sp sp benefits the first being ...,19
4,sp sp sp sp sp sp sp sp sp them so how do you ...,20


In [ ]:
emo_df_test = pd.read_csv("/content/drive/MyDrive/Dissertation/dataset/csv_files/eTest.csv")
display(emo_df_test.head()) 

,Anger,Disgust,Fear,Happy,Sad,Surprise
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
test_df_final = pd.concat([test_df, emo_df_test], axis = 1)
display(test_df_final.head())

,Sentence,Sentence Length,Anger,Disgust,Fear,Happy,Sad,Surprise
0,what doing sp not talking about just going int...,19,0.0,0.0,0.0,1.0,0.0,0.0
1,super connectors that help businesses via thei...,20,0.0,0.0,0.0,1.0,0.0,0.0
2,sp sp sp sp now you see this online with big b...,20,0.0,0.0,0.0,1.0,0.0,0.0
3,sp sp sp sp sp sp sp benefits the first being ...,19,0.0,0.0,0.0,1.0,0.0,0.0
4,sp sp sp sp sp sp sp sp sp them so how do you ...,20,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
test_df_final.drop("Sentence Length", axis=1, inplace=True)
display(test_df_final.head())

,Sentence,Anger,Disgust,Fear,Happy,Sad,Surprise
0,what doing sp not talking about just going int...,0.0,0.0,0.0,1.0,0.0,0.0
1,super connectors that help businesses via thei...,0.0,0.0,0.0,1.0,0.0,0.0
2,sp sp sp sp now you see this online with big b...,0.0,0.0,0.0,1.0,0.0,0.0
3,sp sp sp sp sp sp sp benefits the first being ...,0.0,0.0,0.0,1.0,0.0,0.0
4,sp sp sp sp sp sp sp sp sp them so how do you ...,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
print(test_df_final.shape)

(4832, 7)


In [ ]:
sentences_train = train_df_final.Sentence.values

In [ ]:
emotion_scores_train = train_df_final.iloc[:,1:].values
print(emotion_scores_train)

[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [ ]:
# for i, value in enumerate(emotion_scores):
#   score_list = []
#   for score in value:
#     score_list.append(int(score))
#   emotion_scores[i] = score_list
# print(emotion_scores)

In [ ]:
sentences_valid = valid_df_final.Sentence.values

In [ ]:
emotion_scores_valid = valid_df_final.iloc[:,1:].values
print(emotion_scores_valid)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Testing the tokenizer
test_sent = "Hello! Welcome to Dubai"
test_sent_tokens = tokenizer.tokenize(test_sent)
test_sent_token_ids = tokenizer.convert_tokens_to_ids(test_sent_tokens)

print('Sentence: ', test_sent)
print('Tokens: ', test_sent_tokens)
print('Token IDs: ', test_sent_token_ids)

Sentence:  Hello! Welcome to Dubai
Tokens:  ['hello', '!', 'welcome', 'to', 'dubai']
Token IDs:  [7592, 999, 6160, 2000, 11558]


In [ ]:
tokenizer.sep_token, tokenizer.sep_token_id  

('[SEP]', 102)

In [ ]:
tokenizer.cls_token, tokenizer.cls_token_id

('[CLS]', 101)

In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id

('[PAD]', 0)

In [ ]:
tokenizer.unk_token, tokenizer.unk_token_id

('[UNK]', 100)

In [ ]:
max_len_sent = 0

for text in sentences_train:

    token_ids = tokenizer.encode(text, add_special_tokens=True)
    max_len_sent = max(max_len_sent, len(token_ids))

print('Length of the longest sentence : ', max_len_sent)

Length of the longest sentence :  30


In [ ]:
MAX_LEN = 45
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32

In [ ]:
class emoClass(Dataset):
  def __init__(self, datFrame, sentence, emo_score, tokenizer, max_len):
    self.datFrame = datFrame
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.sent = sentence
    self.emotions = emo_score
  
  def __len__(self):
    return len(self.datFrame)

  def __getitem__(self, index):
    sentence = self.sent[index]

    encoded_sentence = self.tokenizer.encode_plus(
          sentence,
          add_special_tokens = True, 
          max_length = self.max_len,
          pad_to_max_length = True,
          return_attention_mask = True, 
          return_tensors = 'pt',
      )

    input_ids = encoded_sentence['input_ids']
    attention_mask = encoded_sentence['attention_mask']

    return {
            'sentence':sentence,
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'emotion_scores': torch.tensor(self.emotions[index], dtype=torch.float)
        }

In [ ]:
train_dataset = emoClass(train_df_final, sentences_train, emotion_scores_train, tokenizer, MAX_LEN)
print(train_dataset[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'sentence': 'independence we are only bound by human rights that is our standard what we apply in our work we', 'input_ids': tensor([[ 101, 4336, 2057, 2024, 2069, 5391, 2011, 2529, 2916, 2008, 2003, 2256,
         3115, 2054, 2057, 6611, 1999, 2256, 2147, 2057,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'emotion_scores': tensor([0., 1., 0., 0., 0., 0.])}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

In [ ]:
valid_dataset = emoClass(valid_df_final, sentences_valid, emotion_scores_valid, tokenizer, MAX_LEN)
print(valid_dataset[0])

{'sentence': 'sp sp sp sp sp albums now you might wonder why are you buying your favorite albums again i have', 'input_ids': tensor([[  101, 11867, 11867, 11867, 11867, 11867,  4042,  2085,  2017,  2453,
          4687,  2339,  2024,  2017,  9343,  2115,  5440,  4042,  2153,  1045,
          2031,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'emotion_scores': tensor([0., 0., 0., 1., 0., 0.])}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

In [ ]:
loader_train_data = DataLoader(
              train_dataset,
              sampler = torch.utils.data.SequentialSampler(train_dataset),
              batch_size = TRAIN_BATCH_SIZE
          )

In [ ]:
loader_valid_data = DataLoader(
              valid_dataset,
              sampler = torch.utils.data.SequentialSampler(valid_dataset),
              batch_size = VALID_BATCH_SIZE
          )

In [ ]:
import random
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertPreTrainedModel, BertModel

In [ ]:
class MultiLabel_BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(MultiLabel_BertForSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(768, self.config.num_labels)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                position_ids=None, head_mask=None, inputs_embeds=None, labels=None,
                training=True, labeltask1 = None, labeltask2 = None, fine_tune=False):
        
        if fine_tune:
          with torch.no_grad():

            outputs = self.bert(input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids,
                                position_ids=position_ids,
                                head_mask=head_mask,
                                inputs_embeds=inputs_embeds)

            pooled_output = outputs[1]
        else:
            outputs = self.bert(input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids,
                                position_ids=position_ids,
                                head_mask=head_mask,
                                inputs_embeds=inputs_embeds)

            pooled_output = outputs[1]
            last_hidden = outputs[0]


        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,pooled_output) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
          loss_fct = nn.BCEWithLogitsLoss()
          loss = loss_fct(logits, labels)     
          outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
total_emotions = 6

In [ ]:
model = MultiLabel_BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 6, 
    output_attentions = False,
    output_hidden_states = True,
)

model.cuda()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MultiLabel_BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing MultiLabel_BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MultiLabel_BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MultiLabel_BertForSequenceClassification were 

MultiLabel_BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [ ]:
LEARNING_RATE=2e-5    
EPOCHS=4   

In [ ]:
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = 1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from transformers import get_linear_schedule_with_warmup

total_steps = len(loader_train_data) * EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
total_t0 = time.time()

for epoch in range(0, EPOCHS):
  # ========================================
  #               Training
  # ========================================
    
  # Perform one full pass over the training set.

  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch + 1, EPOCHS))
  print('Training...')

  t0 = time.time()

  total_train_loss = 0

  model.train()

  for step, batch in enumerate(loader_train_data):

      if step % 40 == 0 and not step == 0:
          elapsed = format_time(time.time() - t0)
            
          print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(loader_train_data), elapsed))
      
      input_ids = batch["input_ids"].to(device, dtype = torch.long)
      attention_mask = batch["attention_mask"].to(device, dtype = torch.long)
      emotion_scores = batch["emotion_scores"].to(device, dtype = torch.float)

      model.zero_grad()

      input_ids = input_ids.squeeze(1)

      loss, logits, _, _ = model(input_ids, attention_mask= attention_mask,
                             labels=emotion_scores)
      
      total_train_loss += loss.item()

      loss.backward()

      # Clip the norm of the gradients to 1.0.
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Update parameters and take a step using the computed gradient.
      # The optimizer dictates the "update rule"--how the parameters are
      # modified based on their gradients, the learning rate, etc.
      optimizer.step()

      # Update the learning rate.
      scheduler.step()


  # Calculate the average loss over all of the batches.
  avg_train_loss = total_train_loss / len(loader_train_data)            
    
  # Measure how long this epoch took.
  training_time = format_time(time.time() - t0)

  print("")
  print("Average training loss: {0:.2f}".format(avg_train_loss))
  print("Training epcoh took: {:}".format(training_time))

  # ========================================
  #               Validation
  # ========================================
  # After the completion of each training epoch, measure our performance on
  # our validation set.

  print("")
  print("Running Validation...")

  t0 = time.time()

  model.eval()

  total_eval_loss = 0
  nb_eval_steps = 0

  for batch in loader_valid_data:
    input_ids = batch["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch["attention_mask"].to(device, dtype = torch.long)
    emotion_scores = batch["emotion_scores"].to(device, dtype = torch.float)

    input_ids = input_ids.squeeze(1)

    with torch.no_grad():
      loss, logits, _, _ = model(input_ids, attention_mask=attention_mask,
                                 labels=emotion_scores)
      
      total_eval_loss += loss.item()

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = emotion_scores.to('cpu').numpy()

  avg_val_loss = total_eval_loss / len(loader_valid_data)

  if epoch>0:
    if avg_val_loss >= best_loss:
      best_loss = avg_val_loss
      torch.save(model.state_dict(),'/content/drive/MyDrive/Dissertation/dataset/data/finetuned_model.bin')
  else:
    best_loss = avg_val_loss
    torch.save(model.state_dict(),'/content/drive/MyDrive/Dissertation/dataset/data/finetuned_model.bin')

  
  validation_time = format_time(time.time() - t0)
    
  print("Validation Loss: {0:.2f}".format(avg_val_loss))
  print("Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

  Batch    40  of    478.    Elapsed: 0:00:11.
  Batch    80  of    478.    Elapsed: 0:00:22.
  Batch   120  of    478.    Elapsed: 0:00:33.
  Batch   160  of    478.    Elapsed: 0:00:44.
  Batch   200  of    478.    Elapsed: 0:00:55.
  Batch   240  of    478.    Elapsed: 0:01:06.
  Batch   280  of    478.    Elapsed: 0:01:17.
  Batch   320  of    478.    Elapsed: 0:01:28.
  Batch   360  of    478.    Elapsed: 0:01:39.
  Batch   400  of    478.    Elapsed: 0:01:51.
  Batch   440  of    478.    Elapsed: 0:02:02.

Average training loss: 0.32
Training epcoh took: 0:02:12

Running Validation...
Validation Loss: 0.34
Validation took: 0:00:09

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    478.    Elapsed: 0:00:11.
  Batch    80  of    478.    Elapsed: 0:00:22.
  Batch   120  of    478.    Elapsed: 0:00:34.
  Batch   160  of    478.    Elapsed: 0:00:45.
  Batch   200  of    478.    Elapsed: 0:00:56.
  Batch   240  of    478.    Elapsed: 0:01:07.
  Batch   280  of    478.    E

In [ ]:
feature_extractor = MultiLabel_BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = total_emotions, 
    output_attentions = False,
    output_hidden_states = True, 
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MultiLabel_BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing MultiLabel_BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MultiLabel_BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MultiLabel_BertForSequenceClassification were 

In [ ]:
feature_extractor.load_state_dict(torch.load("/content/drive/MyDrive/Dissertation/dataset/data/finetuned_model.bin"))
feature_extractor.cuda()

MultiLabel_BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [ ]:
EPOCHS_NEW = 1

import time
import datetime

words = []
feature_matrix = []
labels = []

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
total_t0 = time.time()

for epoch in range(0, EPOCHS_NEW):
  # ========================================
  #               Training
  # ========================================
    
  # Perform one full pass over the training set.

  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch + 1, EPOCHS_NEW))
  print('Training...')

  # Measure how long the training epoch takes.
  t0 = time.time()

  # Reset the total loss for this epoch.
  total_train_loss = 0

  for step, batch in enumerate(loader_train_data):
    if step % 40 == 0 and not step == 0:
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(loader_train_data), elapsed))

    input_ids = batch["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch["attention_mask"].to(device, dtype = torch.long)
    emotion_scores = batch["emotion_scores"].to(device, dtype = torch.float)

    feature_extractor.zero_grad()

    input_ids = input_ids.squeeze(1)

    with torch.no_grad():
      logits, CLS, hid = feature_extractor(input_ids, attention_mask=attention_mask)

    words.append(hid[0])
    feature_matrix.append(CLS)
    labels.append(emotion_scores)

  print("")
  print("Running Validation...")

  t0 = time.time()

  total_eval_loss = 0
  nb_eval_steps = 0

  for batch in loader_valid_data:

    input_ids = batch["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch["attention_mask"].to(device, dtype = torch.long)
    emotion_scores = batch["emotion_scores"].to(device, dtype = torch.float)

    input_ids = input_ids.squeeze(1)

    with torch.no_grad():
      logits, CLS, hid = feature_extractor(input_ids, attention_mask=attention_mask)
    
    words.append(hid[0]) 
    feature_matrix.append(CLS)
    labels.append(emotion_scores)


======== Epoch 1 / 1 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

  Batch    40  of    478.    Elapsed: 0:00:03.
  Batch    80  of    478.    Elapsed: 0:00:06.
  Batch   120  of    478.    Elapsed: 0:00:10.
  Batch   160  of    478.    Elapsed: 0:00:13.
  Batch   200  of    478.    Elapsed: 0:00:16.
  Batch   240  of    478.    Elapsed: 0:00:20.
  Batch   280  of    478.    Elapsed: 0:00:23.
  Batch   320  of    478.    Elapsed: 0:00:26.
  Batch   360  of    478.    Elapsed: 0:00:30.
  Batch   400  of    478.    Elapsed: 0:00:33.
  Batch   440  of    478.    Elapsed: 0:00:37.

Running Validation...


In [ ]:
sentences_test = test_df_final.Sentence.values

In [ ]:
emotion_scores_test = test_df_final.iloc[:,1:].values
print(emotion_scores_test)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [ ]:
test_dataset = emoClass(test_df_final, sentences_test, emotion_scores_test, tokenizer, MAX_LEN)
print(test_dataset[0])

{'sentence': 'what doing sp not talking about just going into your local starbucks and talking to everybody there i find', 'input_ids': tensor([[  101,  2054,  2725, 11867,  2025,  3331,  2055,  2074,  2183,  2046,
          2115,  2334, 29500,  1998,  3331,  2000,  7955,  2045,  1045,  2424,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'emotion_scores': tensor([0., 0., 0., 1., 0., 0.])}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

In [ ]:
TEST_BATCH_SIZE = 32

In [ ]:
loader_test_data = DataLoader(
              test_dataset,
              sampler = torch.utils.data.SequentialSampler(test_dataset),
              batch_size = TEST_BATCH_SIZE
          )

In [ ]:
feature_extractor.eval()

predictions , true_labels = [], []

for batch in loader_test_data:
  input_ids = batch["input_ids"].to(device, dtype = torch.long)
  attention_mask = batch["attention_mask"].to(device, dtype = torch.long)
  emotion_scores = batch["emotion_scores"].to(device, dtype = torch.float)

  input_ids = input_ids.squeeze(1)

  with torch.no_grad():
    logits,out, hid = feature_extractor(input_ids, attention_mask=attention_mask)
    words.append(hid[0])
    feature_matrix.append(out)
    labels.append(emotion_scores)

  logits = logits.detach().cpu().numpy()
  label_ids = emotion_scores.to('cpu').numpy()

  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

    DONE.


In [ ]:
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr

In [ ]:
flat_predictions = np.concatenate(predictions, axis=0)

flat_predictions = flat_predictions.flatten()

flat_true_labels = np.concatenate(true_labels, axis=0)

In [ ]:
print(flat_true_labels)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [ ]:
print(len(flat_predictions))

28992


In [ ]:
from sklearn import metrics

In [ ]:
score_list = []
for i, value in enumerate(flat_true_labels):
  for score in value:
    score_list.append(int(score))

In [ ]:
print(len(score_list))

28992


In [ ]:
flat_polarities = [int(log>=0) for log in flat_predictions]
true_polarities = score_list

print(len(flat_polarities))
# Calculate the MCC
cc = pearsonr(score_list, flat_predictions)[0]
MAE = mean_absolute_error(score_list, flat_predictions)
acc = metrics.accuracy_score(flat_polarities, true_polarities)
f1 = metrics.f1_score(true_polarities, flat_polarities)
print('Total CC from simple BERT: %.3f' % cc) # --> Double Check the results from the feature extractors correspond to previous ones
print('Total MAE from simple BERT: %.3f' % MAE)
print('Total Acc from simple BERT: %.3f' % acc)
print('Total F1 from simple BERT: %.3f' % f1)

28992
Total CC from simple BERT: 0.438
Total MAE from simple BERT: 2.531
Total Acc from simple BERT: 0.845
Total F1 from simple BERT: 0.395


In [ ]:
import pickle

In [ ]:
DATA = torch.cat(feature_matrix, dim=0).cpu()

In [ ]:
last_hiddens = torch.cat([words[k] for k in range(len(words))],dim=0).cpu()

In [ ]:
labels = [lab.cpu().float() for lab in labels]
lab_tot = torch.cat(labels,dim=0)

In [ ]:
Bert_dict = {'Data':DATA, 'words': last_hiddens,'level':lab_tot}
  
with open('/content/drive/MyDrive/Dissertation/dataset/data/BERT_features.pkl', 'wb') as fp:
  pickle.dump(Bert_dict, fp)

In [ ]:
with open('/content/drive/MyDrive/Dissertation/dataset/data/BERT_features.pkl', 'rb') as fp:
  testDict = pickle.load(fp)

In [ ]:
print(type(testDict))
print(len(testDict))
print(testDict.keys())
print(list(testDict.items())[:5])

<class 'dict'>
3
dict_keys(['Data', 'words', 'level'])
[('Data', tensor([[-0.6896,  0.4095, -0.8756,  ..., -0.0558,  0.5297,  0.6678],
        [-0.6113,  0.4857, -0.7064,  ..., -0.0250,  0.6356,  0.5880],
        [-0.6002,  0.5228, -0.8050,  ..., -0.1901,  0.6196,  0.6203],
        ...,
        [ 0.0092,  0.2866, -0.8131,  ...,  0.6850,  0.7082,  0.0041],
        [-0.1330,  0.4667, -0.6513,  ...,  0.8437,  0.7242,  0.2770],
        [ 0.0011,  0.5065, -0.7625,  ...,  0.7801,  0.7613, -0.0789]])), ('words', tensor([[[ 0.1606, -0.2876, -0.3232,  ..., -0.0352,  0.0288,  0.1569],
         [-0.7390,  0.2383,  0.3255,  ...,  0.6723,  0.6231, -0.2401],
         [-0.9914,  0.4889, -0.7340,  ...,  0.5526,  0.9601, -0.4202],
         ...,
         [ 0.0078, -0.2332,  0.2640,  ..., -0.0560, -0.2933,  0.2840],
         [-0.0727, -0.3865,  0.1692,  ...,  0.0833, -0.3621,  0.2327],
         [ 0.1257, -0.3715,  0.0673,  ...,  0.3249, -0.6364,  0.1751]],

        [[ 0.1606, -0.2876, -0.3232,  ..., -0.0